## Project Lan

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import string
import json

In [2]:
# source dataset https://www.kaggle.com/datasets/uciml/news-aggregator-dataset/data
path = "uci-news-aggregator.csv"
dataframe = pd.read_csv(path)
dataframe.head()

NameError: name 'pd' is not defined

In [3]:
dataframe = dataframe.drop(['STORY', 'HOSTNAME','TIMESTAMP', "URL", "ID", "PUBLISHER"], axis=1)
dataframe.head()

,TITLE,CATEGORY
0,"Fed official says weak data caused by weather,...",b
1,Fed's Charles Plosser sees high bar for change...,b
2,US open: Stocks fall after Fed official hints ...,b
3,"Fed risks falling 'behind the curve', Charles ...",b
4,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,b


In [4]:
dataframe = dataframe.dropna(subset='CATEGORY')
print('Rows:', dataframe.shape[0])

Rows: 422419


In [5]:
dataframe = dataframe.sample(frac=1)
dataframe.isnull().sum()

TITLE       0
CATEGORY    0
dtype: int64

In [6]:
dataframe.shape

(422419, 2)

In [7]:
# Mengambil baris 1 - 10000 saja
dataframe_cut = dataframe.copy()
dataframe_cut = dataframe_cut.iloc[1:10001,:]

dataframe_cut.shape

(10000, 2)

In [8]:
import re
import string

def hapus_karakter(text):
  text = text.lower()
  text = re.sub("\[.*?\]", "", text)
  text = re.sub("\\W", ' ', text)
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?.+>', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation),"", text)
  text = re.sub('\n', '', text)
  text = re.sub('\w*\d\w*', '', text)
  return text

<>:6: SyntaxWarning: invalid escape sequence '\['
<>:8: SyntaxWarning: invalid escape sequence '\S'
<>:12: SyntaxWarning: invalid escape sequence '\w'
<>:6: SyntaxWarning: invalid escape sequence '\['
<>:8: SyntaxWarning: invalid escape sequence '\S'
<>:12: SyntaxWarning: invalid escape sequence '\w'
C:\Users\jmd\AppData\Local\Temp\ipykernel_37940\620876523.py:6: SyntaxWarning: invalid escape sequence '\['
  text = re.sub("\[.*?\]", "", text)
C:\Users\jmd\AppData\Local\Temp\ipykernel_37940\620876523.py:8: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub('https?://\S+|www\.\S+', '', text)
C:\Users\jmd\AppData\Local\Temp\ipykernel_37940\620876523.py:12: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub('\w*\d\w*', '', text)


In [9]:
dataframe_cut['TITLE'] = dataframe_cut['TITLE'].apply(hapus_karakter)
dataframe_cut['CATEGORY'] = dataframe_cut['CATEGORY'].apply(hapus_karakter)

dataframe_cut.head()

,TITLE,CATEGORY
109040,nirvana hall and oates inducted by rock and r...,e
50124,year old shaves head to support friend is su...,m
73805,dating site urges users to ditch firefox,t
191802,china s april crude oil imports at record high,b
247222,mental illness shortens life more than smoking...,m


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np

x = dataframe_cut['TITLE']
y = dataframe_cut['CATEGORY']

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = .25)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

# Pemodelan

## Model Logistic

In [13]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [14]:
LR.score(xv_test, y_test)

0.8628

In [15]:
pred_LR = LR.predict(xv_test)

In [16]:
print(classification_report(y_test,pred_LR))

              precision    recall  f1-score   support

           b       0.85      0.86      0.85       683
           e       0.85      0.96      0.90       934
           m       0.95      0.66      0.78       263
           t       0.88      0.80      0.84       620

    accuracy                           0.86      2500
   macro avg       0.88      0.82      0.84      2500
weighted avg       0.87      0.86      0.86      2500



## Model Decision Tree

In [17]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [18]:
DT.score(xv_test,y_test)

0.7168

In [19]:
pred_DT = DT.predict(xv_test)

In [20]:
print(classification_report(y_test, pred_DT))

              precision    recall  f1-score   support

           b       0.69      0.65      0.67       683
           e       0.72      0.83      0.77       934
           m       0.64      0.57      0.60       263
           t       0.78      0.67      0.72       620

    accuracy                           0.72      2500
   macro avg       0.71      0.68      0.69      2500
weighted avg       0.72      0.72      0.71      2500



## Model Random Forest

In [21]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier() #memanggil model
RF.fit(xv_train, y_train) #training model

RandomForestClassifier()

In [22]:
RF.score(xv_test,y_test)

0.8004

In [23]:
pred_RF = RF.predict(xv_test)

In [24]:
print(classification_report(y_test, pred_DT))

              precision    recall  f1-score   support

           b       0.69      0.65      0.67       683
           e       0.72      0.83      0.77       934
           m       0.64      0.57      0.60       263
           t       0.78      0.67      0.72       620

    accuracy                           0.72      2500
   macro avg       0.71      0.68      0.69      2500
weighted avg       0.72      0.72      0.71      2500



### Compare Model

In [25]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print('----------Accuracy----------')
print('Logistic Regression : {:.2f}%'.format(accuracy_score(y_test, pred_LR)*100))
print('Decision Tree       : {:.2f}%'.format(accuracy_score(y_test, pred_DT)*100))
print('Random Forest       : {:.2f}%'.format(accuracy_score(y_test, pred_RF)*100))

print('============================')
print('============================')

print('---------Precision----------')
print('Logistic Regression : {:.2f}%'.format(precision_score(y_test, pred_LR, average='weighted')*100))
print('Decision Tree       : {:.2f}%'.format(precision_score(y_test, pred_DT, average='weighted')*100))
print('Random Forest       : {:.2f}%'.format(precision_score(y_test, pred_RF, average='weighted')*100))

print('============================')
print('============================')

print('-----------Recall-----------')
print('Logistic Regression : {:.2f}%'.format(recall_score(y_test, pred_LR, average='weighted')*100))
print('Decision Tree       : {:.2f}%'.format(recall_score(y_test, pred_DT, average='weighted')*100))
print('Random Forest       : {:.2f}%'.format(recall_score(y_test, pred_RF, average='weighted')*100))

print('============================')
print('============================')

# Karena hasil dari Precision dan recall tidak seimbang maka dicari f1_score
print('-----------f1_score-----------')
print('============================')
print('Logistic Regression : {:.2f}%'.format(f1_score(y_test, pred_LR, average='weighted')*100))
print('Decision Tree       : {:.2f}%'.format(f1_score(y_test, pred_DT, average='weighted')*100))
print('Random Forest       : {:.2f}%'.format(f1_score(y_test, pred_RF, average='weighted')*100))

----------Accuracy----------
Logistic Regression : 86.28%
Decision Tree       : 71.68%
Random Forest       : 80.04%
---------Precision----------
Logistic Regression : 86.63%
Decision Tree       : 71.73%
Random Forest       : 81.15%
-----------Recall-----------
Logistic Regression : 86.28%
Decision Tree       : 71.68%
Random Forest       : 80.04%
-----------f1_score-----------
Logistic Regression : 86.03%
Decision Tree       : 71.43%
Random Forest       : 79.71%


# Realtime

In [26]:
def output(n):
  if any(n=='b'): # 1 diambil dari dataset
    return "KATEGORI BERITA BUSSINESS"
  elif any(n=='e'):
    return "KATEGORI BERITA ENTERTAIMENT"
  elif any(n=='t'):
    return "KATEGORI BERITA SCIENCE & TECHNOLOGY"
  elif any(n=='m'):
    return "KATEGORI BERITA HEALTH"

def uji_prediksi(title):
  testing_title = {"title" : [title]}
  new_df_test = pd.DataFrame(testing_title)
  new_df_test['title'] = new_df_test['title'].apply(hapus_karakter)
  new_x_test = new_df_test['title']
  new_xv_test = vectorization.transform(new_x_test)
  pred_LR = LR.predict(new_xv_test)
  pred_DT = DT.predict(new_xv_test)
  pred_RF = RF.predict(new_xv_test)

  return print("\n\n\n LR Prediction : {} \n DT Prediction  : {}\n RF Prediction : {}". format(
    output(pred_LR), output(pred_DT), output(pred_RF),
  ))

In [38]:
dataframe['TITLE'].loc[109040]

'Nirvana, Hall and Oates inducted by Rock and Roll Hall of Fame'

In [36]:
dataframe['CATEGORY'].loc[109040]

'e'

In [40]:
title = str(input())

uji_prediksi(title)




 LR Prediction : KATEGORI BERITA ENTERTAIMENT 
 DT Prediction  : KATEGORI BERITA ENTERTAIMENT
 RF Prediction : KATEGORI BERITA ENTERTAIMENT
